# Scraping Nextdoor Neighborhoods

In [ ]:
# Imports
%matplotlib inline
import pandas as pd
import numpy as np
import json
import geojson
import shapefile
import requests
from bs4 import BeautifulSoup
import difflib
import pprint as pp
import six.moves.urllib as urllib
import matplotlib.pyplot as plt
import re
from timeit import default_timer as timer

In [ ]:
state = 'CA'
state_url = 'https://nextdoor.com/find-neighborhood/' + state + '/'

In [ ]:
from requests import get
response = requests.get(state_url)
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

In [ ]:
hood_city_group = html_soup.find_all('div', class_ = 'hood_group')
print(type(hood_city_group))
print(len(hood_city_group))

In [ ]:
df_cities = pd.DataFrame(index=range(5000), columns = ['State', 'City', 'Link'])
df_loc = 0
df_cities.head(1)

In [ ]:
for div in hood_city_group:
    city_links = div.findAll('a')
    for a in city_links:
        df_cities.iloc[df_loc, df_cities.columns.get_loc("State")] = 'CA'
        df_cities.iloc[df_loc, df_cities.columns.get_loc("City")] = a.string
        df_cities.iloc[df_loc, df_cities.columns.get_loc("Link")] = a['href'].strip()
        df_loc += 1

df_cities.dropna(axis = 0, inplace=True)
print(df_loc)

In [ ]:
df_cities.reindex(columns=['State','County','City', 'Link'])
df_cities.head(1)

In [ ]:
df_cities.to_csv('mg_nextdoor_california_cities.csv', index=False)

In [ ]:
cities = ['San Jose','Santa Clara','Sunnyvale', 'Palo Alto','Mountain View',\
          'Cupertino','Milpitas','Los Gatos', 'Gilroy', 'Morgan Hill', 'Campbell',\
          'Los Altos','Saratoga', 'Stanford', 'Los Altos Hills',\
          'San Martin']

In [ ]:
df_nextdoor_neighborhoods = pd.DataFrame(index=range(7000), columns = ['State', 'County', 'City', 'Neighborhood', 'Link'])
df_loc = 0
df_nextdoor_neighborhoods.head(1)

In [ ]:
for city in cities:
    print(city)
    city_lookup = df_cities[df_cities['City'] == city]["Link"].values[0]
    print(city_lookup)
    response = requests.get(city_lookup)
    responseTxt = response.text.encode('UTF-8')
    html_soup = BeautifulSoup(responseTxt, 'html.parser')
    type(html_soup) 
    hood_neighborhood_group = html_soup.find_all('div', class_ = 'hood_group')
    print(type(hood_neighborhood_group))
    print(len(hood_neighborhood_group))
   
    for div in hood_neighborhood_group:
        neighborhood_links = div.findAll('a')
        for a in neighborhood_links:
            print(city + " " + a.string + " " +  a['href'])
            df_nextdoor_neighborhoods.iloc[df_loc, df_nextdoor_neighborhoods.columns.get_loc("State")] = 'CA'
            df_nextdoor_neighborhoods.iloc[df_loc, df_nextdoor_neighborhoods.columns.get_loc("County")] = 'Santa Clara'
            df_nextdoor_neighborhoods.iloc[df_loc, df_nextdoor_neighborhoods.columns.get_loc("City")] = city
            df_nextdoor_neighborhoods.iloc[df_loc, df_nextdoor_neighborhoods.columns.get_loc("Neighborhood")] = str(a.string.encode('UTF-8'))
            df_nextdoor_neighborhoods.iloc[df_loc, df_nextdoor_neighborhoods.columns.get_loc("Link")] = str(a['href'].strip())
            df_loc += 1

df_nextdoor_neighborhoods.dropna(axis = 0, inplace=True)
print(df_loc)

In [ ]:
df_nextdoor_neighborhoods.head(3)

In [ ]:
df_nextdoor_neighborhoods.to_csv('mg_nextdoor_neighborhoods_master.csv', index=False)

In [ ]:
df_nextdoor_neighborhoods = pd.read_csv("mg_nextdoor_neighborhood_fulllist.csv")
df_nextdoor_neighborhoods.head(1)

In [ ]:
df_nextdoor_neighborhoods["Nextdoor ID"] = 0
df_nextdoor_neighborhoods["Geometry"] = ""
df_nextdoor_neighborhoods["Interests"] = ""
df_nextdoor_neighborhoods["Gardening Interest"] = 0
df_nextdoor_neighborhoods["Percentage of Homeowners"] = 0
df_nextdoor_neighborhoods["Number of Residents"] = 0
df_nextdoor_neighborhoods["Average Age"] = 0
df_nextdoor_neighborhoods['Attributes'] = ""

In [ ]:
df_nextdoor_neighborhoods["Link"].fillna("", inplace=True)
df_nextdoor_neighborhoods["Geometry"].fillna("", inplace=True)
df_nextdoor_neighborhoods["Interests"].fillna("", inplace=True)
df_nextdoor_neighborhoods["Attributes"].fillna("", inplace=True)

In [ ]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches

In [ ]:
def findAllTheNearbyNeighborhoods(neighborhood_info):
    print("findAllTheNearbyNeighborhoods")
    global df_nextdoor_neighborhoods
    nearbyNeighborhoods = neighborhood_info[neighborhood_info.index('nearbyNeighborhoods: '): neighborhood_info.index('neighborhoodGeometriesJSON:')]
    nearbyNeighborhoods = nearbyNeighborhoods.lstrip()
    nearbyNeighborhoods = nearbyNeighborhoods.rstrip()
    begIndex = len('nearbyNeighborhoods: ')
    nearbyNeighborhoods = nearbyNeighborhoods[begIndex: -1]
    nearby_neighborhood_json = json.loads(nearbyNeighborhoods)
    for nearby_neighborhood in nearby_neighborhood_json:
        if (nearby_neighborhood['city'] in cities):
            nearby_neighborhood['shortName'] = re.sub('\s+', ' ', nearby_neighborhood['shortName']).strip()
            no_entry = df_nextdoor_neighborhoods.loc[df_nextdoor_neighborhoods['Neighborhood'] == nearby_neighborhood['shortName']]
            print(nearby_neighborhood)
            if (len(no_entry) == 0):
                print("Appending: ", nearby_neighborhood)
                df_nextdoor_neighborhoods = df_nextdoor_neighborhoods.append(
                    {"State": nearby_neighborhood['state'],
                     "County": "Santa Clara",
                     "City": nearby_neighborhood['city'],
                     "Neighborhood": nearby_neighborhood['shortName'],
                     "Link": "",
                     "Nextdoor ID": 0,
                     "Geometry": "",
                     "Interests": "",
                     "Gardening Interest": 0,
                     "Percentage of Homeowners": 0,
                     "Number of Residents": 0,
                     "Average Age": 0,
                     "Attributes": ""
                    }, ignore_index=True)
    print("findAllTheNearbyNeighborhoods")

In [ ]:
# Find the id of the neighborhood the page is for
def findTheNeighborhoodID(currentRowIndex, neighborhood_info):
    print("findTheNeighborhoodID")
    global df_nextdoor_neighborhoods
    neighborhoodMap = neighborhood_info[neighborhood_info.index('neighborhoodMapOptions:{'): -1]
    neighborhoodMap = neighborhoodMap.lstrip()
    neighborhoodMap = neighborhoodMap.rstrip()
    begIDIndex = list(find_all(neighborhoodMap, "hoodId: "))   
    assert(len(begIDIndex) == 1)
    endIDIndex = begIDIndex[0] + len ("hoodId: ") + 10
    hoodId = neighborhoodMap[begIDIndex[0] + len ("hoodId: "): endIDIndex]
    hoodId = hoodId.lstrip()
    hoodId = hoodId.rstrip()
    print(hoodId)
    hoodId = re.findall(r'^([\s\d]+)$', hoodId)
    print(hoodId)
    assert(len(hoodId) == 1)
    df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Nextdoor ID")] = int(hoodId[0])
    print("findTheNeighborhoodID")

In [ ]:
# Iterate over each neighborhood and update the URL and ID
def findAllTheNeighborhoodIDs(neighborhood_info):
    print("findAllTheNeighborhoodIDs")
    global df_nextdoor_neighborhoods
    neighborhoodMap = neighborhood_info[neighborhood_info.index('neighborhoodMapOptions:{'): -1]
    neighborhoodMap = neighborhoodMap.lstrip()
    neighborhoodMap = neighborhoodMap.rstrip()
    print(neighborhoodMap)
    begURLIndexes = list(find_all(neighborhoodMap, "\"page_url\": \""))  
    endURLIndexes = list(find_all(neighborhoodMap, "\", \"short_name\": ")) 
    print(begURLIndexes)
    print(endURLIndexes)
    begShortNameIndexes = list(find_all(neighborhoodMap, "\"short_name\": \"")) 
    endShortNameIndexes = list(find_all(neighborhoodMap, "\", \"stroke_color\": ")) 
    print(begShortNameIndexes)
    print(endShortNameIndexes)
    begIDIndexes = list(find_all(neighborhoodMap, "\"id\": ")) 
    endIDIndexes = list(find_all(neighborhoodMap, ", \"geometry\": \"{")) 
    print(begIDIndexes)
    print(endIDIndexes)
    if ((len(begURLIndexes) == len(endURLIndexes)) and 
        (len(begShortNameIndexes) == len(endShortNameIndexes)) and 
        (len(begIDIndexes) == len(endIDIndexes))):
        for begShort, endShort, begURL, endURL, begID, endID in zip(begShortNameIndexes, endShortNameIndexes, 
                                                               begURLIndexes, endURLIndexes, 
                                                               begIDIndexes, endIDIndexes):
            print(begShort, endShort)
            print(begURL, endURL)
            print(begID, endID)
            shortname = neighborhoodMap[begShort + len ("\"short_name\": \""): endShort]
            shortname = re.sub('\s+', ' ', shortname).strip()
            print(shortname)
            entry = df_nextdoor_neighborhoods.loc[df_nextdoor_neighborhoods['Neighborhood'] == shortname]
            if (len(entry) == 1):
                pp.pprint(entry)
                index = int(df_nextdoor_neighborhoods.index[df_nextdoor_neighborhoods['Neighborhood'] == shortname][0])
                print(df_nextdoor_neighborhoods.iloc[index])
                if (len(df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("Link")]) == 0):
                    page_url = neighborhoodMap[begURL + len ("\"page_url\": \""): endURL]
                    print("Updating Page URL: " + page_url)
                    if (len(page_url) > 0):
                        df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("Link")] = page_url
                if (df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("Nextdoor ID")] == 0): 
                    neighborhood_id = neighborhoodMap[begID + len ("\"id\": "): endID]
                    print("ID: " + neighborhood_id)
                    df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("Nextdoor ID")] = int(neighborhood_id)
                print(df_nextdoor_neighborhoods.iloc[index])
                print("findAllTheNeighborhoodIDs loop")                                
    print("findAllTheNeighborhoodIDs")

In [ ]:
# Iterate over each feature and update the row for that neighborhood name
def findAllTheNeighborhoodFeatures(neighborhood_info):
    print("findAllTheNeighborhoodFeatures")
    global df_nextdoor_neighborhoods
    neighborhoodGeometries = neighborhood_info[neighborhood_info.index('neighborhoodGeometriesJSON: '): neighborhood_info.index('neighborhoodMapOptions:{')]
    neighborhoodGeometries = neighborhoodGeometries.lstrip()
    neighborhoodGeometries = neighborhoodGeometries.rstrip()
    begIndex = len('neighborhoodGeometriesJSON: ')
    neighborhoodGeometries = neighborhoodGeometries[begIndex: -1]
    neighborhood_json = json.loads(neighborhoodGeometries)
    for feature in neighborhood_json["features"]:
        props = feature["properties"]
        props["hood_name"] = re.sub('\s+', ' ', props['hood_name']).strip()
        print("Name: %s, ID: %s \n" % (props["hood_name"], props["hood_id"]))
        entry = df_nextdoor_neighborhoods.loc[(df_nextdoor_neighborhoods['Neighborhood'] == props["hood_name"]) & 
                                              (df_nextdoor_neighborhoods['Nextdoor ID'] == props["hood_id"])]
        print("Entry: " + str(entry))
        if (len(entry) == 1):
            index = int(df_nextdoor_neighborhoods.index[df_nextdoor_neighborhoods['Nextdoor ID'] == props["hood_id"]][0])
            print("Entry at Index: " + str(df_nextdoor_neighborhoods.iloc[index]))
            #Save the feature for that neighborhood without the featurecollection 
            print("Geometry: " + str(df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("Geometry")]))
            if (len(df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("Geometry")]) == 0):
                feature['properties']['State'] = df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("State")]
                feature['properties']['City'] = df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("City")]
                feature['properties']['County'] = "Santa Clara"
                feature['properties']['Neighborhood'] = props["hood_name"]
                feature['properties']['Nextdoor ID'] = props["hood_id"]
                del feature['properties']['hood_name']
                del feature['properties']['hood_id']
                del feature['properties']['fill']
                del feature['properties']['stroke']
                del feature['properties']['fill-opacity']
                df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("Geometry")] = json.dumps(feature)
            if (df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("Nextdoor ID")] == 0): 
                df_nextdoor_neighborhoods.iat[index, df_nextdoor_neighborhoods.columns.get_loc("Nextdoor ID")] = props["hood_id"]
            print(df_nextdoor_neighborhoods.iloc[index])
        print("findAllTheNeighborhoodFeatures loop")
    print("findAllTheNeighborhoodFeatures")

In [ ]:
def iterateOneNeighborhood(currentRowIndex):
    neighborhood_info = ""
    if len(df_nextdoor_neighborhoods.iloc[currentRowIndex]["Link"]) > 0:
        neighborhood_lookup = df_nextdoor_neighborhoods.iloc[currentRowIndex]["Link"]
        response = requests.get(neighborhood_lookup)
        responseTxt = response.text.encode('UTF-8')
        html_soup = BeautifulSoup(responseTxt, 'html.parser')
     
        try:
            # find the variable with all the neighborhood info
            scripts = html_soup.findAll('script')
            neighborhood_info = scripts[5].string.strip()
            neighborhood_info = neighborhood_info[neighborhood_info.index('['):-1]
            
            findTheNeighborhoodID(currentRowIndex, neighborhood_info)
            pp.pprint(df_nextdoor_neighborhoods.iloc[currentRowIndex])
        
            findAllTheNearbyNeighborhoods(neighborhood_info)
            pp.pprint(df_nextdoor_neighborhoods.iloc[currentRowIndex])

            findAllTheNeighborhoodIDs(neighborhood_info)
            pp.pprint(df_nextdoor_neighborhoods.iloc[currentRowIndex])

            findAllTheNeighborhoodFeatures(neighborhood_info)
            pp.pprint(df_nextdoor_neighborhoods.iloc[currentRowIndex])

            # Update with the interest info
            try:
                begInterestIndex = neighborhood_info.index('interests: [')
                endInterestIndex = neighborhood_info.index('iosUrl: "https://')
                if ((begInterestIndex > 0) and (endInterestIndex > 0)):
                    interests = neighborhood_info[begInterestIndex: endInterestIndex]
                    interests = interests[interests.index('[')+1: interests.index(']')]
                    interests = interests.replace('"', "")
                    interest_list = interests.split(", ")
                    print(interest_list)
                    df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Interests")] = interest_list
                    if ('Gardening & Landscape' in interest_list):
                        df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Gardening Interest")] = interest_list.index('Gardening & Landscape') + 1
            except:
                print("No interests found for: ", df_nextdoor_neighborhoods.iloc[currentRowIndex]["Neighborhood"])
        
            # Update with the attribute info
            try:
                begAttrIndex = neighborhood_info.index('attributes: [')
                endAttrIndex = neighborhood_info.index('census:')
                if ((begAttrIndex > 0) and (endAttrIndex > 0)):
                    attributes = neighborhood_info[begAttrIndex: endAttrIndex]
                    attributes = attributes[attributes.index('[')+1: attributes.index(']')]
                    attributes = attributes.replace('"', "")
                    attributes_list = attributes.split(", ")
                    print(attributes_list)
                    df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Attributes")] = attributes_list
            except:
                print("No attributes found for: ", df_nextdoor_neighborhoods.iloc[currentRowIndex]["Neighborhood"])
           
            # Update with the census info
            try:
                begCensusIndex = neighborhood_info.index('census: {')
                endCensusIndex = neighborhood_info.index('city: ')
                if ((begCensusIndex > 0) and (endCensusIndex > 0)):
                    census = neighborhood_info[begCensusIndex: endCensusIndex]
                    census = census[census.index('{'): census.index('}')+1]
                    census = json.loads(census)
                    print("Census: %s\n" % str(census))
                    df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Percentage of Homeowners")] = int(census['homeowners'])
                    df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Number of Residents")] = int(census['population'])
                    df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Average Age")] = int(census['age'])
            except:
                print("No census found for: ", df_nextdoor_neighborhoods.iloc[currentRowIndex]["Neighborhood"])
        
        except:
            print("Error parsing")
    return (neighborhood_info)

In [ ]:
def iterateOneNeighborhoodInterest(currentRowIndex):
    neighborhood_info = ""
    if len(df_nextdoor_neighborhoods.iloc[currentRowIndex]["Link"]) > 0:
        neighborhood_lookup = df_nextdoor_neighborhoods.iloc[currentRowIndex]["Link"]
        response = requests.get(neighborhood_lookup)
        responseTxt = response.text.encode('UTF-8')
        html_soup = BeautifulSoup(responseTxt, 'html.parser')
     
        try:
            # find the variable with all the neighborhood info
            scripts = html_soup.findAll('script')
            neighborhood_info = scripts[5].string.strip()
            neighborhood_info = neighborhood_info[neighborhood_info.index('['):-1]

            # Update with the interest info
            try:
                begInterestIndex = neighborhood_info.index('interests: [')
                endInterestIndex = neighborhood_info.index('iosUrl: "https://')
                if ((begInterestIndex > 0) and (endInterestIndex > 0)):
                    interests = neighborhood_info[begInterestIndex: endInterestIndex]
                    interests = interests[interests.index('[')+1: interests.index(']')]
                    interests = interests.replace('"', "")
                    interest_list = interests.split(", ")
                    print(interest_list)
                    df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Interests")] = interest_list
                    if ('Gardening & Landscape' in interest_list):
                        df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Interest in Gardening")] = interest_list.index('Gardening & Landscape')
                    else:
                        df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Interest in Gardening")] = 10
            except:
                print("No interests found for: ", df_nextdoor_neighborhoods.iloc[currentRowIndex]["Neighborhood"])
        except:
            print("Error parsing")
    return (neighborhood_info)

In [ ]:
currentIndex = 0
endIndex = len(df_nextdoor_neighborhoods) - 1
print("Number of Neighborhoods: " + str(len(df_nextdoor_neighborhoods)))
print("End Index: " , str(endIndex))
while True:
    print("Current Index: " + str(currentIndex))
    iterateOneNeighborhoodInterest(currentIndex) 
    currentIndex += 1
    if currentIndex > endIndex:
        break

In [ ]:
def iterateOneNeighborhoodActivity(currentRowIndex):
    neighborhood_info = ""
    if len(df_nextdoor_neighborhoods.iloc[currentRowIndex]["Link"]) > 0:
        neighborhood_lookup = df_nextdoor_neighborhoods.iloc[currentRowIndex]["Link"]
        response = requests.get(neighborhood_lookup)
        responseTxt = response.text.encode('UTF-8')
        html_soup = BeautifulSoup(responseTxt, 'html.parser')
     
        try:
            # find the variable with all the neighborhood info
            scripts = html_soup.findAll('script')
            neighborhood_info = scripts[5].string.strip()
            neighborhood_info = neighborhood_info[neighborhood_info.index('['):-1]

            try:
                if df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Attributes")] == "":
                    print("Updating Attributes")
                    begAttrIndex = neighborhood_info.index('attributes: [')
                    endAttrIndex = neighborhood_info.index('census:')
                    if ((begAttrIndex > 0) and (endAttrIndex > 0)):
                        attributes = neighborhood_info[begAttrIndex: endAttrIndex]
                        attributes = attributes[attributes.index('[')+1: attributes.index(']')]
                        attributes = attributes.replace('"', "")
                        attributes_list = attributes.split(", ")
                        print(attributes_list)
                        df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Attributes")] = attributes_list
            except:
                print("No attributes found for: ", df_nextdoor_neighborhoods.iloc[currentRowIndex]["Neighborhood"])
           
            # Update with the census info
            try:
                    print("Updating Census")
                    begCensusIndex = neighborhood_info.index('census: {')
                    endCensusIndex = neighborhood_info.index('city: ')
                    if ((begCensusIndex > 0) and (endCensusIndex > 0)):
                        census = neighborhood_info[begCensusIndex: endCensusIndex]
                        census = census[census.index('{'): census.index('}')+1]
                        census = json.loads(census)
                        print("Census: %s\n" % str(census))
                        df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Percentage of Homeowners")] = int(census['homeowners'])
                        df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Number of Residents")] = int(census['population'])
                        df_nextdoor_neighborhoods.iat[currentRowIndex, df_nextdoor_neighborhoods.columns.get_loc("Average Age")] = int(census['age'])
            except:
                print("No census found for: ", df_nextdoor_neighborhoods.iloc[currentRowIndex]["Neighborhood"])
        
        except:
            print("Error parsing")
    return (neighborhood_info)

In [ ]:
currentIndex = 0
endIndex = len(df_nextdoor_neighborhoods) - 1
print("Number of Neighborhoods: " + str(len(df_nextdoor_neighborhoods)))
print("End Index: " , str(endIndex))
while True:
    print("Current Index: " + str(currentIndex))
    iterateOneNeighborhoodActivity(currentIndex) 
    currentIndex += 1
    if currentIndex > endIndex:
        break

In [ ]:
df_nextdoor_neighborhoods.to_csv('nextdoor_with_properties_master.csv', index=False)

In [ ]:
interest_columns = ['State', 'County', 'City', 'Neighborhood', "Nextdoor ID", "Interest", "Position"]
attribute_columns = ['State', 'County', 'City', 'Neighborhood', "Nextdoor ID", "Attribute", "Position"]
df_nextdoor_interests = pd.DataFrame(columns=interest_columns)
df_nextdoor_attributes = pd.DataFrame(columns=attribute_columns)

In [ ]:
def iterateOneNeighborhoodInterestActivity(currentRowIndex):
    global df_nextdoor_interests
    global df_nextdoor_attributes
    entry = df_nextdoor_neighborhoods.iloc[currentRowIndex]
    interest_list = entry["Interests"] 
    print(interest_list)
    for idx, interest in enumerate(interest_list):
        df_nextdoor_interests = df_nextdoor_interests.append(
                    {"State": entry['State'],
                     "County": entry['County'],
                     "City": entry['City'],
                     "Neighborhood": entry['Neighborhood'],
                     "Nextdoor ID": entry['Nextdoor ID'],
                     "Interest": interest,
                     "Position": idx
                    }, ignore_index=True)
        
    attribute_list = entry["Attributes"] 
    print(attribute_list)
    for idx, attribute in enumerate(attribute_list):
        df_nextdoor_attributes = df_nextdoor_attributes.append(
                    {"State": entry['State'],
                     "County": entry['County'],
                     "City": entry['City'],
                     "Neighborhood": entry['Neighborhood'],
                     "Nextdoor ID": entry['Nextdoor ID'],
                     "Attribute": attribute,
                     "Position": idx
                    }, ignore_index=True)   

In [ ]:
currentIndex = 0
endIndex = len(df_nextdoor_neighborhoods) - 1
print("Number of Neighborhoods: " + str(len(df_nextdoor_neighborhoods)))
print("End Index: " , str(endIndex))
while True:
    print("Current Index: " + str(currentIndex))
    iterateOneNeighborhoodInterestActivity(currentIndex) 
    currentIndex += 1
    if currentIndex > endIndex:
        break

In [ ]:
# Resave neighborhood information to csv files for input to Tableau
df_nextdoor_interests.to_csv('nextdoor_interests_master.csv', index=False)
df_nextdoor_attributes.to_csv('nextdoor_attributes_master.csv', index=False)

In [ ]:
# Save geographic information to geojson file for input to Tableau
from geojson import Point, Feature, FeatureCollection, dumps

features = []
for index, row in df_nextdoor_neighborhoods.iterrows():
    if (len(row["Geometry"]) > 0):
        features.append(json.loads(row["Geometry"]))

feature_collection = FeatureCollection(features)

with open('nextdoor_neighborhoods.geojson', 'w') as f:
   dump(feature_collection, f)